<a href="https://colab.research.google.com/github/RiverBotham/Raman/blob/main/Raman%20Imaging%20Denoising%20-%20TransUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO:


*   Test TransUNet1D
*   If any good try adding it to ResUNet



In [1]:
# To save forst clone the repo
!git config --global user.name "RiverBotham"
!git config --global user.email "river.botham@gmail.com"
!git config --global user.password "MY_PASSWORD"

token = 'MY_TOKEN'
username = 'RiverBotham'
repo = 'Raman'

!git clone https://{token}@github.com/{username}/{repo}

Cloning into 'Raman'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 107 (delta 60), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 7.50 MiB | 7.89 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [2]:
# Move into the cloned repo, then File -> Save copy in GitHub
%cd {repo}/Denoising

/content/Raman/Denoising


In [3]:
# Imports
import os
import sys
import random
import datetime
import time
import shutil
import argparse
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import scipy.signal
import math
from skimage.metrics import structural_similarity as sk_ssim
from sklearn.model_selection import KFold

import torch
from torch import nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributed as dist
import torch.utils.data.distributed
import torch.multiprocessing as mp
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torch.utils.tensorboard import SummaryWriter
import torch.cuda.amp as amp
from torchvision import transforms, utils
import pandas as pd
import matplotlib.pyplot as plt
from einops import rearrange, repeat

# import model, dataset, utilities

In [ ]:
# class MultiHeadAttention(nn.Module):
#     def __init__(self, embedding_dim, head_num):
#         super().__init__()

#         self.head_num = head_num
#         self.dk = (embedding_dim // head_num) ** (1 / 2)

#         self.qkv_layer = nn.Linear(embedding_dim, embedding_dim * 3, bias=False)
#         self.out_attention = nn.Linear(embedding_dim, embedding_dim, bias=False)

#     def forward(self, x, mask=None):
#         qkv = self.qkv_layer(x)

#         query, key, value = tuple(rearrange(qkv, 'b t (d k h) -> k b h t d', k=3, h=self.head_num))
#         energy = torch.einsum("... i d, ... j d -> ... i j", query, key) * self.dk

#         if mask is not None:
#             energy = energy.masked_fill(mask, -float('inf'))

#         attention = torch.softmax(energy, dim=-1)

#         x = torch.einsum("... i j, ... j d -> ... i d", attention, value)

#         x = rearrange(x, "b h t d -> b t (h d)")
#         x = self.out_attention(x)

#         return x


# class MLP(nn.Module):
#     def __init__(self, embedding_dim, mlp_dim):
#         super().__init__()

#         self.mlp_layers = nn.Sequential(
#             nn.Linear(embedding_dim, mlp_dim),
#             nn.GELU(),
#             nn.Dropout(0.1),
#             nn.Linear(mlp_dim, embedding_dim),
#             nn.Dropout(0.1)
#         )

#     def forward(self, x):
#         return self.mlp_layers(x)


# class TransformerEncoderBlock(nn.Module):
#     def __init__(self, embedding_dim, head_num, mlp_dim):
#         super().__init__()

#         self.multi_head_attention = MultiHeadAttention(embedding_dim, head_num)
#         self.mlp = MLP(embedding_dim, mlp_dim)

#         self.layer_norm1 = nn.LayerNorm(embedding_dim)
#         self.layer_norm2 = nn.LayerNorm(embedding_dim)

#         self.dropout = nn.Dropout(0.1)

#     def forward(self, x):
#         _x = self.multi_head_attention(x)
#         _x = self.dropout(_x)
#         x = x + _x
#         x = self.layer_norm1(x)

#         _x = self.mlp(x)
#         x = x + _x
#         x = self.layer_norm2(x)

#         return x


# class Transformer1D(nn.Module):
#     def __init__(self, input_dim, embedding_dim, head_num, mlp_dim, block_num):
#         super().__init__()

#         self.embedding = nn.Linear(input_dim, embedding_dim)  # Embedding for 1D input
#         self.transformer_blocks = nn.ModuleList([
#             TransformerEncoderBlock(embedding_dim, head_num, mlp_dim) for _ in range(block_num)
#         ])

#     def forward(self, x):
#         # x shape: (batch_size, seq_length, input_dim)
#         x = self.embedding(x)  # Shape: (batch_size, seq_length, embedding_dim)
#         for block in self.transformer_blocks:
#             x = block(x)
#         return x


# class EncoderBottleneck(nn.Module):
#     def __init__(self, in_channels, out_channels, stride=1, base_width=64):
#         super().__init__()

#         self.downsample = nn.Sequential(
#             nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
#             nn.BatchNorm1d(out_channels)
#         )

#         width = int(out_channels * (base_width / 64))

#         self.conv1 = nn.Conv1d(in_channels, width, kernel_size=1, stride=1, bias=False)
#         self.norm1 = nn.BatchNorm1d(width)

#         self.conv2 = nn.Conv1d(width, width, kernel_size=3, stride=2, padding=1, bias=False)
#         self.norm2 = nn.BatchNorm1d(width)

#         self.conv3 = nn.Conv1d(width, out_channels, kernel_size=1, stride=1, bias=False)
#         self.norm3 = nn.BatchNorm1d(out_channels)

#         self.relu = nn.ReLU(inplace=True)

#     def forward(self, x):
#         x_down = self.downsample(x)

#         x = self.conv1(x)
#         x = self.norm1(x)
#         x = self.relu(x)

#         x = self.conv2(x)
#         x = self.norm2(x)
#         x = self.relu(x)

#         x = self.conv3(x)
#         x = self.norm3(x)
#         x = x + x_down
#         x = self.relu(x)

#         return x


# class DecoderBottleneck(nn.Module):
#     def __init__(self, in_channels, out_channels, scale_factor=2):
#         super().__init__()

#         self.upsample = nn.Upsample(scale_factor=scale_factor, mode='linear', align_corners=True)
#         self.layer = nn.Sequential(
#             nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm1d(out_channels),
#             nn.ReLU(inplace=True),
#             nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm1d(out_channels),
#             nn.ReLU(inplace=True)
#         )

#     def forward(self, x, x_concat=None):
#         x = self.upsample(x)

#         if x_concat is not None:
#             x = torch.cat([x_concat, x], dim=1)

#         x = self.layer(x)
#         return x


# class Encoder(nn.Module):
#     def __init__(self, signal_length, in_channels, out_channels, head_num, mlp_dim, block_num, patch_dim):
#         super().__init__()

#         self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=7, stride=2, padding=3, bias=False)
#         self.norm1 = nn.BatchNorm1d(out_channels)
#         self.relu = nn.ReLU(inplace=True)

#         self.encoder1 = EncoderBottleneck(out_channels, out_channels * 2, stride=2)
#         self.encoder2 = EncoderBottleneck(out_channels * 2, out_channels * 4, stride=2)
#         self.encoder3 = EncoderBottleneck(out_channels * 4, out_channels * 8, stride=2)

#         self.vit_signal_length = signal_length // patch_dim
#         self.vit = Transformer1D(self.vit_signal_length, out_channels * 8, head_num, mlp_dim, block_num)

#         self.conv2 = nn.Conv1d(out_channels * 8, 512, kernel_size=3, stride=1, padding=1)
#         self.norm2 = nn.BatchNorm1d(512)

#     def forward(self, x):
#         x = self.conv1(x)  # Shape: (batch_size, out_channels, reduced_length)
#         x = self.norm1(x)
#         x1 = self.relu(x)

#         x2 = self.encoder1(x1)
#         x3 = self.encoder2(x2)
#         x = self.encoder3(x3)

#         # Reshape for transformer input
#         x = rearrange(x, 'b c l -> b l c')  # Change to (batch_size, length, channels)
#         x = self.vit(x)  # Pass through the transformer
#         x = rearrange(x, 'b l c -> b c l')  # Change back to (batch_size, channels, length)

#         x = self.conv2(x)
#         x = self.norm2(x)
#         x = self.relu(x)

#         return x, x1, x2, x3


# class Decoder(nn.Module):
#     def __init__(self, out_channels, class_num):
#         super().__init__()

#         self.decoder1 = DecoderBottleneck(out_channels * 8, out_channels * 4)
#         self.decoder2 = DecoderBottleneck(out_channels * 4, out_channels * 2)
#         self.decoder3 = DecoderBottleneck(out_channels * 2, int(out_channels * 1 / 2))
#         self.decoder4 = DecoderBottleneck(int(out_channels * 1 / 2), int(out_channels * 1 / 8))

#         self.conv1 = nn.Conv1d(int(out_channels * 1 / 8), class_num, kernel_size=1)

#     def forward(self, x, x1, x2, x3):
#         x = self.decoder1(x, x3)
#         x = self.decoder2(x, x2)
#         x = self.decoder3(x, x1)
#         x = self.decoder4(x)
#         x = self.conv1(x)

#         return x


# class TransUNet(nn.Module):
#     def __init__(self, signal_length, in_channels, out_channels, head_num, mlp_dim, block_num, patch_dim, class_num):
#         super().__init__()

#         self.encoder = Encoder(signal_length, in_channels, out_channels,
#                                head_num, mlp_dim, block_num, patch_dim)

#         self.decoder = Decoder(out_channels, class_num)

#     def forward(self, x):
#         x, x1, x2, x3 = self.encoder(x)
#         x = self.decoder(x, x1, x2, x3)

#         return x

In [ ]:
# # Trans U net

# import torch
# import torch.nn as nn
# import numpy as np
# from einops import rearrange, repeat

# # Multi-Head Attention for 1D
# class MultiHeadAttention(nn.Module):
#     def __init__(self, embedding_dim, head_num):
#         super().__init__()

#         self.head_num = head_num
#         self.dk = (embedding_dim // head_num) ** (1 / 2)

#         self.qkv_layer = nn.Linear(embedding_dim, embedding_dim * 3, bias=False)
#         self.out_attention = nn.Linear(embedding_dim, embedding_dim, bias=False)

#     def forward(self, x, mask=None):
#         qkv = self.qkv_layer(x)

#         query, key, value = tuple(rearrange(qkv, 'b t (d k h ) -> k b h t d ', k=3, h=self.head_num))
#         energy = torch.einsum("... i d , ... j d -> ... i j", query, key) * self.dk

#         if mask is not None:
#             energy = energy.masked_fill(mask, -np.inf)

#         attention = torch.softmax(energy, dim=-1)

#         x = torch.einsum("... i j , ... j d -> ... i d", attention, value)

#         x = rearrange(x, "b h t d -> b t (h d)")
#         x = self.out_attention(x)

#         return x


# class MLP(nn.Module):
#     def __init__(self, embedding_dim, mlp_dim):
#         super().__init__()

#         self.mlp_layers = nn.Sequential(
#             nn.Linear(embedding_dim, mlp_dim),
#             nn.GELU(),
#             nn.Dropout(0.1),
#             nn.Linear(mlp_dim, embedding_dim),
#             nn.Dropout(0.1)
#         )

#     def forward(self, x):
#         x = self.mlp_layers(x)

#         return x


# class TransformerEncoderBlock(nn.Module):
#     def __init__(self, embedding_dim, head_num, mlp_dim):
#         super().__init__()

#         self.multi_head_attention = MultiHeadAttention(embedding_dim, head_num)
#         self.mlp = MLP(embedding_dim, mlp_dim)

#         self.layer_norm1 = nn.LayerNorm(embedding_dim)
#         self.layer_norm2 = nn.LayerNorm(embedding_dim)

#         self.dropout = nn.Dropout(0.1)

#     def forward(self, x):
#         _x = self.multi_head_attention(x)
#         _x = self.dropout(_x)
#         x = x + _x
#         x = self.layer_norm1(x)

#         _x = self.mlp(x)
#         x = x + _x
#         x = self.layer_norm2(x)

#         return x


# class TransformerEncoder(nn.Module):
#     def __init__(self, embedding_dim, head_num, mlp_dim, block_num=12):
#         super().__init__()

#         self.layer_blocks = nn.ModuleList(
#             [TransformerEncoderBlock(embedding_dim, head_num, mlp_dim) for _ in range(block_num)])

#     def forward(self, x):
#         for layer_block in self.layer_blocks:
#             x = layer_block(x)

#         return x


# class ViT(nn.Module):
#     def __init__(self, seq_len, in_channels, embedding_dim, head_num, mlp_dim,
#                  block_num, patch_dim, classification=False, num_classes=1):
#         super().__init__()

#         self.patch_dim = patch_dim
#         self.classification = classification
#         self.num_tokens = (seq_len // patch_dim) ** 2
#         self.token_dim = in_channels * (patch_dim ** 2)

#         self.projection = nn.Linear(self.token_dim, embedding_dim)
#         self.embedding = nn.Parameter(torch.rand(self.num_tokens + 1, embedding_dim))

#         self.cls_token = nn.Parameter(torch.randn(1, 1, embedding_dim))

#         self.dropout = nn.Dropout(0.1)

#         self.transformer = TransformerEncoder(embedding_dim, head_num, mlp_dim, block_num)

#         if self.classification:
#             self.mlp_head = nn.Linear(embedding_dim, num_classes)

#     def forward(self, x):
#         seq_patches = rearrange(x,
#                                 'b c (patch s) -> b (s) (patch c)',
#                                 patch=self.patch_dim)

#         batch_size, tokens, _ = seq_patches.shape

#         project = self.projection(seq_patches)
#         token = repeat(self.cls_token, 'b ... -> (b batch_size) ...',
#                        batch_size=batch_size)

#         patches = torch.cat([token, project], dim=1)
#         patches += self.embedding[:tokens + 1, :]

#         x = self.dropout(patches)
#         x = self.transformer(x)
#         x = self.mlp_head(x[:, 0, :]) if self.classification else x[:, 1:, :]

#         return x



# class EncoderBottleneck(nn.Module):
#     def __init__(self, in_channels, out_channels, stride=1, base_width=64):
#         super().__init__()

#         self.downsample = nn.Sequential(
#             nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
#             nn.BatchNorm1d(out_channels)
#         )

#         width = int(out_channels * (base_width / 64))

#         self.conv1 = nn.Conv1d(in_channels, width, kernel_size=1, stride=1, bias=False)
#         self.norm1 = nn.BatchNorm1d(width)

#         self.conv2 = nn.Conv1d(width, width, kernel_size=3, stride=2, groups=1, padding=1, dilation=1, bias=False)
#         self.norm2 = nn.BatchNorm1d(width)

#         self.conv3 = nn.Conv1d(width, out_channels, kernel_size=1, stride=1, bias=False)
#         self.norm3 = nn.BatchNorm1d(out_channels)

#         self.relu = nn.ReLU(inplace=True)

#     def forward(self, x):
#         x_down = self.downsample(x)

#         x = self.conv1(x)
#         x = self.norm1(x)
#         x = self.relu(x)

#         x = self.conv2(x)
#         x = self.norm2(x)
#         x = self.relu(x)

#         x = self.conv3(x)
#         x = self.norm3(x)
#         x = x + x_down
#         x = self.relu(x)

#         return x


# class DecoderBottleneck(nn.Module):
#     def __init__(self, in_channels, out_channels, scale_factor=2):
#         super().__init__()

#         self.upsample = nn.Upsample(scale_factor=scale_factor, mode='linear', align_corners=True)
#         self.layer = nn.Sequential(
#             nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm1d(out_channels),
#             nn.ReLU(inplace=True),
#             nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm1d(out_channels),
#             nn.ReLU(inplace=True)
#         )

#     def forward(self, x, x_concat=None):
#         x = self.upsample(x)

#         if x_concat is not None:
#             print(f"x_concat shape: {x_concat.shape}")
#             print(f"x shape: {x.shape}")
#             x = torch.cat([x_concat, x], dim=1)

#         x = self.layer(x)
#         return x


# class Encoder(nn.Module):
#     def __init__(self, seq_len, in_channels, out_channels, head_num, mlp_dim, block_num, patch_dim):
#         super().__init__()

#         self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=7, stride=2, padding=3, bias=False)
#         self.norm1 = nn.BatchNorm1d(out_channels)
#         self.relu = nn.ReLU(inplace=True)

#         self.encoder1 = EncoderBottleneck(out_channels, out_channels * 2, stride=2)
#         self.encoder2 = EncoderBottleneck(out_channels * 2, out_channels * 4, stride=2)
#         self.encoder3 = EncoderBottleneck(out_channels * 4, out_channels * 8, stride=2)

#         self.vit_seq_len = seq_len // patch_dim
#         self.vit = ViT(self.vit_seq_len, out_channels * 8, out_channels * 8,
#                        head_num, mlp_dim, block_num, patch_dim=1, classification=False)

#         self.conv2 = nn.Conv1d(out_channels * 8, 512, kernel_size=3, stride=1, padding=1)
#         self.norm2 = nn.BatchNorm1d(512)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.norm1(x)
#         x1 = self.relu(x)

#         x2 = self.encoder1(x1)
#         x3 = self.encoder2(x2)
#         x = self.encoder3(x3)

#         x = self.vit(x)
#         x = rearrange(x, "b c l -> b l c")

#         x = self.conv2(x)
#         x = self.norm2(x)
#         x = self.relu(x)

#         return x, x1, x2, x3


# class Decoder(nn.Module):
#     def __init__(self, out_channels, class_num):
#         super().__init__()

#         self.decoder1 = DecoderBottleneck(out_channels * 8, out_channels * 2)
#         self.decoder2 = DecoderBottleneck(out_channels * 4, out_channels)
#         self.decoder3 = DecoderBottleneck(out_channels * 2, int(out_channels * 1 / 2))
#         self.decoder4 = DecoderBottleneck(int(out_channels * 1 / 2), int(out_channels * 1 / 8))

#         self.conv1 = nn.Conv1d(int(out_channels * 1 / 8), class_num, kernel_size=1)

#     def forward(self, x, x1, x2, x3):
#         print(f"Initial x1 shape: {x1.shape}, x2 shape: {x2.shape}")
#         print(f"Initial x shape: {x.shape}, x3 shape: {x3.shape}")
#         x = self.decoder1(x, x3)
#         print(f"After decoder 1 x shape: {x.shape}, x2 shape: {x2.shape}")
#         x = self.decoder2(x, x2)
#         print(f"After decoder 2 x shape: {x.shape}, x1 shape: {x1.shape}")
#         x = self.decoder3(x, x1)
#         print(f"After decoder 3 x shape: {x.shape}")
#         x = self.decoder4(x)
#         print(f"Final decoder output shape before conv1: {x.shape}")
#         x = self.conv1(x)

#         return x


# class TransUNet(nn.Module):
#     def __init__(self, img_dim, in_channels, out_channels, head_num, mlp_dim, block_num, patch_dim, class_num):
#         super().__init__()

#         self.encoder = Encoder(img_dim, in_channels, out_channels,
#                                head_num, mlp_dim, block_num, patch_dim)

#         self.decoder = Decoder(out_channels, class_num)

#     def forward(self, x):
#         x, x1, x2, x3 = self.encoder(x)
#         x = self.decoder(x, x1, x2, x3)

#         return x

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from einops import rearrange, repeat

# # Multi-Head Attention for 1D
# class MultiHeadAttention(nn.Module):
#     def __init__(self, embedding_dim, head_num):
#         super().__init__()
#         self.head_num = head_num
#         self.dk = (embedding_dim // head_num) ** (1 / 2)
#         self.qkv_layer = nn.Linear(embedding_dim, embedding_dim * 3, bias=False)
#         self.out_attention = nn.Linear(embedding_dim, embedding_dim, bias=False)

#     def forward(self, x, mask=None):
#         qkv = self.qkv_layer(x)
#         query, key, value = tuple(rearrange(qkv, 'b t (d k h) -> k b h t d', k=3, h=self.head_num))
#         energy = torch.einsum("... i d , ... j d -> ... i j", query, key) * self.dk

#         if mask is not None:
#             energy = energy.masked_fill(mask, float('-inf'))

#         attention = torch.softmax(energy, dim=-1)
#         x = torch.einsum("... i j , ... j d -> ... i d", attention, value)
#         x = rearrange(x, "b h t d -> b t (h d)")
#         x = self.out_attention(x)

#         return x

# # MLP for Feedforward
# class MLP(nn.Module):
#     def __init__(self, embedding_dim, mlp_dim):
#         super().__init__()
#         self.mlp_layers = nn.Sequential(
#             nn.Linear(embedding_dim, mlp_dim),
#             nn.GELU(),
#             nn.Dropout(0.1),
#             nn.Linear(mlp_dim, embedding_dim),
#             nn.Dropout(0.1)
#         )

#     def forward(self, x):
#         return self.mlp_layers(x)

# # Transformer Encoder Block
# class TransformerEncoderBlock(nn.Module):
#     def __init__(self, embedding_dim, head_num, mlp_dim):
#         super().__init__()
#         self.multi_head_attention = MultiHeadAttention(embedding_dim, head_num)
#         self.mlp = MLP(embedding_dim, mlp_dim)
#         self.layer_norm1 = nn.LayerNorm(embedding_dim)
#         self.layer_norm2 = nn.LayerNorm(embedding_dim)
#         self.dropout = nn.Dropout(0.1)

#     def forward(self, x):
#         _x = self.multi_head_attention(x)
#         _x = self.dropout(_x)
#         x = x + _x
#         x = self.layer_norm1(x)
#         _x = self.mlp(x)
#         x = x + _x
#         x = self.layer_norm2(x)
#         return x

# # Transformer Encoder
# class TransformerEncoder(nn.Module):
#     def __init__(self, embedding_dim, head_num, mlp_dim, block_num=12):
#         super().__init__()
#         self.layer_blocks = nn.ModuleList(
#             [TransformerEncoderBlock(embedding_dim, head_num, mlp_dim) for _ in range(block_num)]
#         )

#     def forward(self, x):
#         for layer_block in self.layer_blocks:
#             x = layer_block(x)
#         return x

# # Vision Transformer
# class ViT(nn.Module):
#     def __init__(self, seq_len, in_channels, embedding_dim, head_num, mlp_dim, block_num, patch_dim):
#         super().__init__()
#         self.patch_dim = patch_dim
#         self.num_tokens = (seq_len // patch_dim)
#         self.token_dim = in_channels * (patch_dim)

#         self.projection = nn.Linear(self.token_dim, embedding_dim)
#         self.embedding = nn.Parameter(torch.rand(self.num_tokens + 1, embedding_dim))
#         self.cls_token = nn.Parameter(torch.randn(1, 1, embedding_dim))
#         self.dropout = nn.Dropout(0.1)
#         self.transformer = TransformerEncoder(embedding_dim, head_num, mlp_dim, block_num)

#     def forward(self, x):
#         seq_patches = rearrange(x, 'b c (patch s) -> b (s) (patch c)', patch=self.patch_dim)
#         batch_size, tokens, _ = seq_patches.shape
#         project = self.projection(seq_patches)
#         token = repeat(self.cls_token, 'b ... -> (b batch_size) ...', batch_size=batch_size)

#         patches = torch.cat([token, project], dim=1)
#         patches += self.embedding[:tokens + 1, :]
#         x = self.dropout(patches)
#         x = self.transformer(x)
#         return x[:, 1:, :]  # Return the patch tokens only

# # Encoder Bottleneck
# class EncoderBottleneck(nn.Module):
#     def __init__(self, in_channels, out_channels, stride=1):
#         super().__init__()
#         self.downsample = nn.Sequential(
#             nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
#             nn.BatchNorm1d(out_channels)
#         )

#         self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=2, padding=1, bias=False)
#         self.norm1 = nn.BatchNorm1d(out_channels)

#         self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
#         self.norm2 = nn.BatchNorm1d(out_channels)

#         self.relu = nn.ReLU(inplace=True)

#     def forward(self, x):
#         x_down = self.downsample(x)

#         x = self.conv1(x)
#         x = self.norm1(x)
#         x = self.relu(x)

#         x = self.conv2(x)
#         x = self.norm2(x)
#         x = x + x_down
#         x = self.relu(x)

#         return x

# # Encoder
# class Encoder(nn.Module):
#     def __init__(self, spectrum_len, in_channels, out_channels, head_num, mlp_dim, block_num, patch_dim):
#         super().__init__()
#         self.bottleneck1 = EncoderBottleneck(in_channels, out_channels)        # Output shape will be [batch_size, out_channels, spectrum_len / 2]
#         self.bottleneck2 = EncoderBottleneck(out_channels, out_channels * 2)  # Output shape will be [batch_size, out_channels * 2, spectrum_len / 4]
#         self.bottleneck3 = EncoderBottleneck(out_channels * 2, out_channels * 4)  # Output shape will be [batch_size, out_channels * 4, spectrum_len / 8]

#         self.transformer = ViT(spectrum_len // 8, out_channels * 4, out_channels * 4, head_num, mlp_dim, block_num, patch_dim)  # Adjusting input dimensions for ViT

#     def forward(self, x):
#         print(f"Input shape: {x.shape}")

#         x1 = self.bottleneck1(x)
#         print(f"After bottleneck1 shape: {x1.shape}")

#         x2 = self.bottleneck2(x1)
#         print(f"After bottleneck2 shape: {x2.shape}")

#         x3 = self.bottleneck3(x2)
#         print(f"After bottleneck3 shape: {x3.shape}")

#         # Preparing for ViT
#         x3_reshaped = rearrange(x3, 'b c t -> b t c')  # Reshape for ViT input
#         print(f"Reshaped x3 for ViT: {x3_reshaped.shape}")

#         x3_transformed = self.transformer(x3_reshaped)  # Pass through the ViT
#         print(f"Output shape from ViT: {x3_transformed.shape}")

#         return x3_transformed, x1, x2, x3

# # Decoder Bottleneck
# class DecoderBottleneck(nn.Module):
#     def __init__(self, in_channels, out_channels, scale_factor=2):
#         super().__init__()
#         self.upsample = nn.Upsample(scale_factor=scale_factor, mode='linear', align_corners=True)
#         self.layer = nn.Sequential(
#             nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm1d(out_channels),
#             nn.ReLU(inplace=True),
#             nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm1d(out_channels),
#             nn.ReLU(inplace=True)
#         )

#     def forward(self, x, x_concat=None):
#         x = self.upsample(x)
#         if x_concat is not None:
#             # Ensure sizes match before concatenation
#             if x_concat.size(2) != x.size(2):
#                 diff = x_concat.size(2) - x.size(2)
#                 x_concat = F.pad(x_concat, (0, -diff, 0, 0))  # Pad x_concat
#             x = torch.cat((x, x_concat), dim=1)
#         return self.layer(x)

# # Decoder
# class Decoder(nn.Module):
#     def __init__(self, out_channels, num_classes):
#         super().__init__()
#         self.decoder3 = DecoderBottleneck(out_channels * 4, out_channels * 2)
#         self.decoder2 = DecoderBottleneck(out_channels * 2 * 2, out_channels)  # *2 for concatenation
#         self.decoder1 = DecoderBottleneck(out_channels * 2, out_channels)  # *2 for concatenation
#         self.final_conv = nn.Conv1d(out_channels, num_classes, kernel_size=1)

#     def forward(self, x, x1, x2, x3):
#         print(f"Input to decoder: {x.shape}")
#         x = self.decoder3(x, x3)
#         print(f"After decoder3 shape: {x.shape}")
#         x = self.decoder2(x, x2)
#         print(f"After decoder2 shape: {x.shape}")
#         x = self.decoder1(x, x1)
#         print(f"After decoder1 shape: {x.shape}")
#         return self.final_conv(x)

# # Full TransUNet Model
# class TransUNet(nn.Module):
#     def __init__(self, spectrum_len, in_channels, out_channels, head_num, mlp_dim, block_num, num_classes, patch_dim):
#         super().__init__()
#         self.encoder = Encoder(spectrum_len, in_channels, out_channels, head_num, mlp_dim, block_num, patch_dim)
#         self.decoder = Decoder(out_channels, num_classes)

#     def forward(self, x):
#         x, x1, x2, x3 = self.encoder(x)
#         return self.decoder(x, x1, x2, x3)

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class MultiHeadSelfAttention(nn.Module):
#     def __init__(self, embed_dim, num_heads):
#         super(MultiHeadSelfAttention, self).__init__()
#         self.num_heads = num_heads
#         self.head_dim = embed_dim // num_heads

#         assert (
#             self.head_dim * num_heads == embed_dim
#         ), "Embedding dimension must be divisible by number of heads"

#         self.q_linear = nn.Linear(embed_dim, embed_dim)
#         self.k_linear = nn.Linear(embed_dim, embed_dim)
#         self.v_linear = nn.Linear(embed_dim, embed_dim)
#         self.fc_out = nn.Linear(embed_dim, embed_dim)

#     def forward(self, x):
#         N, seq_length, embed_dim = x.shape
#         q = self.q_linear(x).view(N, seq_length, self.num_heads, self.head_dim).transpose(1, 2)  # (N, num_heads, seq_length, head_dim)
#         k = self.k_linear(x).view(N, seq_length, self.num_heads, self.head_dim).transpose(1, 2)  # (N, num_heads, seq_length, head_dim)
#         v = self.v_linear(x).view(N, seq_length, self.num_heads, self.head_dim).transpose(1, 2)  # (N, num_heads, seq_length, head_dim)

#         # Calculate attention scores
#         energy = torch.einsum("nqhd,nkhd->nqkhd", [q, k])  # (N, num_heads, seq_length, seq_length)
#         attention = F.softmax(energy / (self.head_dim ** 0.5), dim=3)

#         out = torch.einsum("nqkhd,nkhd->nqhd", [attention, v]).reshape(N, seq_length, embed_dim)
#         return self.fc_out(out)

# class EncoderBlock(nn.Module):
#     def __init__(self, in_channels, embed_dim):
#         super(EncoderBlock, self).__init__()
#         self.conv = nn.Conv1d(in_channels, embed_dim, kernel_size=3, padding=1)
#         self.attn = MultiHeadSelfAttention(embed_dim, num_heads=4)
#         self.pool = nn.MaxPool1d(2)

#     def forward(self, x):
#         x = F.relu(self.conv(x))  # Convolution layer
#         x = x.permute(0, 2, 1)  # Change to (B, L, C) for attention
#         x = self.attn(x)  # Apply Multi-Head Self-Attention
#         x = x.permute(0, 2, 1)  # Change back to (B, C, L)
#         skip = x  # Skip connection
#         x = self.pool(x)
#         return x, skip

# class DecoderBlock(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(DecoderBlock, self).__init__()
#         self.upconv = nn.ConvTranspose1d(in_channels, out_channels, kernel_size=2, stride=2)
#         self.conv = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)

#     def forward(self, x, skip):
#         x = self.upconv(x)

#         # Adjust the size of skip if necessary
#         if x.size(1) != skip.size(1):  # Check number of channels
#             # Create a Conv1d layer with the same dtype as skip
#             conv_adjust = nn.Conv1d(skip.size(1), x.size(1), kernel_size=1).to(skip.dtype)
#             skip = conv_adjust(skip)  # Adjust channels with 1x1 convolution

#         if x.size(2) != skip.size(2):  # Check length
#             skip = skip[:, :, :x.size(2)]  # Trim the skip connection if necessary

#         x = x + skip  # Skip connection
#         x = F.relu(self.conv(x))
#         return x

# class TransUNetBasic(nn.Module):
#     def __init__(self, in_channels=1, out_channels=1):
#         super(TransUNetBasic, self).__init__()
#         self.encoder1 = EncoderBlock(in_channels, 64)
#         self.encoder2 = EncoderBlock(64, 128)
#         self.encoder3 = EncoderBlock(128, 256)

#         self.decoder3 = DecoderBlock(256, 128)
#         self.decoder2 = DecoderBlock(128, 64)
#         self.decoder1 = DecoderBlock(64, out_channels)

#     def forward(self, x):
#         enc1, skip1 = self.encoder1(x)
#         enc2, skip2 = self.encoder2(enc1)
#         enc3, skip3 = self.encoder3(enc2)

#         dec3 = self.decoder3(enc3, skip3)
#         dec2 = self.decoder2(dec3, skip2)
#         dec1 = self.decoder1(dec2, skip1)

#         return dec1

In [ ]:

# class PatchEmbedding1D(nn.Module):
#     def __init__(self, in_channels, patch_size, embed_dim):
#         super(PatchEmbedding1D, self).__init__()
#         self.patch_size = patch_size
#         self.conv = nn.Conv1d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

#     def forward(self, x):
#         # x: (batch_size, in_channels, signal_length)
#         x = self.conv(x)  # (batch_size, embed_dim, signal_length // patch_size)
#         x = x.permute(0, 2, 1)  # (batch_size, num_patches, embed_dim)
#         return x

# class TransformerEncoderLayer1D(nn.Module):
#     def __init__(self, embed_dim, num_heads, dropout=0.1, dim_feedforward=2048):
#         super(TransformerEncoderLayer1D, self).__init__()
#         self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
#         self.linear1 = nn.Linear(embed_dim, dim_feedforward)
#         self.dropout = nn.Dropout(dropout)
#         self.linear2 = nn.Linear(dim_feedforward, embed_dim)
#         self.norm1 = nn.LayerNorm(embed_dim)
#         self.norm2 = nn.LayerNorm(embed_dim)
#         self.dropout1 = nn.Dropout(dropout)
#         self.dropout2 = nn.Dropout(dropout)

#     def forward(self, src):
#         # src: (num_patches, batch_size, embed_dim)
#         src2 = self.self_attn(src, src, src)[0]
#         src = src + self.dropout1(src2)
#         src = self.norm1(src)
#         src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
#         src = src + self.dropout2(src2)
#         src = self.norm2(src)
#         return src

# class TransformerEncoder1D(nn.Module):
#     def __init__(self, embed_dim, num_heads, depth, dropout=0.1):
#         super(TransformerEncoder1D, self).__init__()
#         self.layers = nn.ModuleList([
#             TransformerEncoderLayer1D(embed_dim, num_heads, dropout) for _ in range(depth)
#         ])

#     def forward(self, x):
#         # x: (batch_size, num_patches, embed_dim)
#         x = x.permute(1, 0, 2)  # (num_patches, batch_size, embed_dim)
#         for layer in self.layers:
#             x = layer(x)
#         x = x.permute(1, 0, 2)  # (batch_size, num_patches, embed_dim)
#         return x

# class UpSampleBlock1D(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(UpSampleBlock1D, self).__init__()
#         self.conv = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
#         self.up = nn.ConvTranspose1d(out_channels, out_channels, kernel_size=2, stride=2)

#     def forward(self, x):
#         x = F.relu(self.conv(x))
#         x = self.up(x)
#         return x

# class TransUNet1D(nn.Module):
#     def __init__(self, in_channels, out_channels, embed_dim, num_heads, depth, patch_size):
#         super(TransUNet1D, self).__init__()
#         self.patch_embed = PatchEmbedding1D(in_channels, patch_size, embed_dim)
#         self.transformer_encoder = TransformerEncoder1D(embed_dim, num_heads, depth)
#         self.decoder1 = UpSampleBlock1D(embed_dim, embed_dim // 2)
#         self.decoder2 = UpSampleBlock1D(embed_dim // 2, embed_dim // 4)
#         self.output_conv = nn.Conv1d(embed_dim // 4, out_channels, kernel_size=1)
#         self.up_final = nn.ConvTranspose1d(16, 16, kernel_size=2, stride=2, padding=150)

#     def forward(self, x):
#         print(f"Input size: {x.shape}")
#         # x: (batch_size, in_channels, signal_length)
#         x = self.patch_embed(x)  # (batch_size, num_patches, embed_dim)
#         print(f"after embedding, expect (batch_size, num_patches, embed_dim) size: {x.shape}")
#         x = self.transformer_encoder(x)  # (batch_size, num_patches, embed_dim)
#         print(f"after transformer_encoder, expect (batch_size, num_patches, embed_dim) size: {x.shape}")

#         # Reshape back to 1D format for the decoder
#         batch_size, num_patches, embed_dim = x.shape
#         x = x.permute(0, 2, 1)  # (batch_size, embed_dim, num_patches)
#         print(f"after permute, expect (batch_size, embed_dim, num_patches) size: {x.shape}")
#         x = F.interpolate(x, scale_factor=2, mode='linear', align_corners=False)  # Upsample
#         print(f"after interpolate size: {x.shape}")

#         # Decoder: Upsample and reconstruct the signal
#         x = self.decoder1(x)  # (batch_size, embed_dim // 2, upsampled_length)
#         print(f"after decoder1 expect (batch_size, embed_dim // 2, upsampled_length) size: {x.shape}")
#         x = self.decoder2(x)  # (batch_size, embed_dim // 4, further_upsampled_length)
#         print(f"after decoder2 expect (batch_size, embed_dim // 4, further_upsampled_length) size: {x.shape}")
#         x = self.up_final(x)
#         print(f"after up_final expect (batch_size, 16, original_length) size: {x.shape}")
#         x = self.output_conv(x)  # (batch_size, out_channels, original_length)
#         print(f"after output_conv expect (batch_size, out_channels, original_length) size: {x.shape}")

#         return x

In [4]:
class PatchEmbedding1D(nn.Module):
    def __init__(self, in_channels, patch_size, embed_dim):
        super(PatchEmbedding1D, self).__init__()
        self.patch_size = patch_size
        self.conv = nn.Conv1d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        # x: (batch_size, in_channels, signal_length)
        x = self.conv(x)  # (batch_size, embed_dim, signal_length // patch_size)
        x = x.permute(0, 2, 1)  # (batch_size, num_patches, embed_dim)
        return x

class TransformerEncoderLayer1D(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout=0.1, dim_feedforward=2048):
        super(TransformerEncoderLayer1D, self).__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
        self.linear1 = nn.Linear(embed_dim, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, embed_dim)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src):
        # src: (num_patches, batch_size, embed_dim)
        src2 = self.self_attn(src, src, src)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

class TransformerEncoder1D(nn.Module):
    def __init__(self, embed_dim, num_heads, depth, dropout=0.1):
        super(TransformerEncoder1D, self).__init__()
        self.layers = nn.ModuleList([
            TransformerEncoderLayer1D(embed_dim, num_heads, dropout) for _ in range(depth)
        ])

    def forward(self, x):
        # x: (batch_size, num_patches, embed_dim)
        x = x.permute(1, 0, 2)  # (num_patches, batch_size, embed_dim)
        for layer in self.layers:
            x = layer(x)
        x = x.permute(1, 0, 2)  # (batch_size, num_patches, embed_dim)
        return x

class UpSampleBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpSampleBlock1D, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.up = nn.ConvTranspose1d(out_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x):
        x = F.relu(self.conv(x))
        x = self.up(x)
        return x

class TransUNet1D(nn.Module):
    def __init__(self, in_channels, out_channels, embed_dim, num_heads, depth, patch_size):
        super(TransUNet1D, self).__init__()
        self.patch_embed = PatchEmbedding1D(in_channels, patch_size, embed_dim)
        self.transformer_encoder = TransformerEncoder1D(embed_dim, num_heads, depth)
        self.decoder1 = UpSampleBlock1D(embed_dim, embed_dim // 2)
        self.decoder2 = UpSampleBlock1D(embed_dim // 2, embed_dim // 4)
        self.output_conv = nn.Conv1d(embed_dim // 4, out_channels, kernel_size=1)

    def forward(self, x):
        # print(f"Input size: {x.shape}")
        # x: (batch_size, in_channels, signal_length)
        x = self.patch_embed(x)  # (batch_size, num_patches, embed_dim)
        # print(f"after embedding, expect (batch_size, num_patches, embed_dim) size: {x.shape}")
        x = self.transformer_encoder(x)  # (batch_size, num_patches, embed_dim)
        # print(f"after transformer_encoder, expect (batch_size, num_patches, embed_dim) size: {x.shape}")

        # Reshape back to 1D format for the decoder
        batch_size, num_patches, embed_dim = x.shape
        x = x.permute(0, 2, 1)  # (batch_size, embed_dim, num_patches)
        # print(f"after permute, expect (batch_size, embed_dim, num_patches) size: {x.shape}")
        x = F.interpolate(x, scale_factor=2, mode='linear', align_corners=False)  # Upsample
        # print(f"after interpolate size: {x.shape}")

        # Decoder: Upsample and reconstruct the signal
        x = self.decoder1(x)  # (batch_size, embed_dim // 2, upsampled_length)
        # print(f"after decoder1 expect (batch_size, embed_dim // 2, upsampled_length) size: {x.shape}")
        x = self.decoder2(x)  # (batch_size, embed_dim // 4, further_upsampled_length)
        # print(f"after decoder2 expect (batch_size, embed_dim // 4, further_upsampled_length) size: {x.shape}")
        x = F.interpolate(x, size=500, mode='linear', align_corners=True)
        # print(f"after up_final expect (batch_size, 16, original_length) size: {x.shape}")
        x = self.output_conv(x)  # (batch_size, out_channels, original_length)
        # print(f"after output_conv expect (batch_size, out_channels, original_length) size: {x.shape}")

        return x

In [5]:
# data set

class RamanDataset(Dataset):
    def __init__(self, inputs, outputs, batch_size=64,spectrum_len=500, spectrum_shift=0.,
                 spectrum_window=False, horizontal_flip=False, mixup=False):
        self.inputs = inputs
        self.outputs = outputs
        self.batch_size = batch_size
        self.spectrum_len = spectrum_len
        self.spectrum_shift = spectrum_shift
        self.spectrum_window = spectrum_window
        self.horizontal_flip = horizontal_flip
        self.mixup = mixup
        self.on_epoch_end()

    def pad_spectrum(self, input_spectrum, spectrum_length):
        if len(input_spectrum) == spectrum_length:
            padded_spectrum = input_spectrum
        elif len(input_spectrum) > spectrum_length:
            padded_spectrum = input_spectrum[0:spectrum_length]
        else:
            padded_spectrum = np.pad(input_spectrum, ((0,spectrum_length - len(input_spectrum)),(0,0)), 'reflect')

        return padded_spectrum

    def window_spectrum(self, input_spectrum, start_idx, window_length):
        if len(input_spectrum) <= window_length:
            output_spectrum = input_spectrum
        else:
            end_idx = start_idx + window_length
            output_spectrum = input_spectrum[start_idx:end_idx]

        return output_spectrum

    def flip_axis(self, x, axis):
        if np.random.random() < 0.5:
            x = np.asarray(x).swapaxes(axis, 0)
            x = x[::-1, ...]
            x = x.swapaxes(0, axis)
        return x

    def shift_spectrum(self, x, shift_range):
        x = np.expand_dims(x,axis=-1)
        shifted_spectrum = x
        spectrum_shift_range = int(np.round(shift_range*len(x)))
        if spectrum_shift_range > 0:
            shifted_spectrum = np.pad(x[spectrum_shift_range:,:], ((0,abs(spectrum_shift_range)), (0,0)), 'reflect')
        elif spectrum_shift_range < 0:
            shifted_spectrum = np.pad(x[:spectrum_shift_range,:], ((abs(spectrum_shift_range), 0), (0,0)), 'reflect')
        return shifted_spectrum

    def mixup_spectrum(self, input_spectrum1, input_spectrum2, output_spectrum1, output_spectrum2, alpha):
        lam = np.random.beta(alpha, alpha)
        input_spectrum = (lam * input_spectrum1) + ((1 - lam) * input_spectrum2)
        output_spectrum = (lam * output_spectrum1) + ((1 - lam) * output_spectrum2)
        return input_spectrum, output_spectrum

    def __getitem__(self, index):
        input_spectrum = self.inputs[index]
        output_spectrum = self.outputs[index]

        mixup_on = False
        if self.mixup:
            if np.random.random() < 0.5:
                spectrum_idx = int(np.round(np.random.random() * (len(self.inputs)-1)))
                input_spectrum2 = self.inputs[spectrum_idx]
                output_spectrum2 = self.outputs[spectrum_idx]
                mixup_on = True

        if self.spectrum_window:
            start_idx = int(np.floor(np.random.random() * (len(input_spectrum)-self.spectrum_len)))
            input_spectrum = self.window_spectrum(input_spectrum, start_idx, self.spectrum_len)
            output_spectrum = self.window_spectrum(output_spectrum, start_idx, self.spectrum_len)
            if mixup_on:
                input_spectrum2 = self.window_spectrum(input_spectrum2, start_idx, self.spectrum_len)
                output_spectrum2 = self.window_spectrum(output_spectrum2, start_idx, self.spectrum_len)

        input_spectrum = self.pad_spectrum(input_spectrum, self.spectrum_len)
        output_spectrum = self.pad_spectrum(output_spectrum, self.spectrum_len)
        if mixup_on:
            input_spectrum2 = self.pad_spectrum(input_spectrum2, self.spectrum_len)
            output_spectrum2 = self.pad_spectrum(output_spectrum2, self.spectrum_len)

        if self.spectrum_shift != 0.0:
            shift_range = np.random.uniform(-self.spectrum_shift, self.spectrum_shift)
            input_spectrum = self.shift_spectrum(input_spectrum, shift_range)
            output_spectrum = self.shift_spectrum(output_spectrum, shift_range)
            if mixup_on:
                input_spectrum2 = self.shift_spectrum(input_spectrum2, shift_range)
                output_spectrum2 = self.shift_spectrum(output_spectrum2, shift_range)
        else:
            input_spectrum = np.expand_dims(input_spectrum, axis=-1)
            output_spectrum = np.expand_dims(output_spectrum, axis=-1)
            if mixup_on:
                input_spectrum2 = np.expand_dims(input_spectrum2, axis=-1)
                output_spectrum2 = np.expand_dims(output_spectrum2, axis=-1)

        if self.horizontal_flip:
            if np.random.random() < 0.5:
                input_spectrum = self.flip_axis(input_spectrum, 0)
                output_spectrum = self.flip_axis(output_spectrum, 0)
                if mixup_on:
                    input_spectrum2 = self.flip_axis(input_spectrum2, 0)
                    output_spectrum2 = self.flip_axis(output_spectrum2, 0)

        if mixup_on:
            input_spectrum, output_spectrum = self.mixup_spectrum(input_spectrum, input_spectrum2, output_spectrum, output_spectrum2, 0.2)

        input_spectrum = input_spectrum/np.amax(input_spectrum)
        output_spectrum = output_spectrum/np.amax(output_spectrum)

        input_spectrum = np.moveaxis(input_spectrum, -1, 0)
        output_spectrum = np.moveaxis(output_spectrum, -1, 0)

        sample = {'input_spectrum': input_spectrum, 'output_spectrum': output_spectrum}

        return sample

    def on_epoch_end(self):
        pass

    def __len__(self):
        return len(self.inputs)

In [6]:
# utilities

class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [7]:
def train(dataloader, net, optimizer, scheduler, criterion, criterion_MSE, epoch, args):

    # For measuring time and losses
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss MSE', ':.4e')
    progress = ProgressMeter(len(dataloader), [batch_time, losses], prefix="Epoch: [{}]".format(epoch))

    # Initialize GradScaler for automatic mixed precision
    scaler = amp.GradScaler()

    end = time.time()

    # Iterate through the batches
    for i, data in enumerate(dataloader):
        # Efficient data transfer to GPU
        inputs = data['input_spectrum'].float().cuda(args.gpu, non_blocking=True)
        target = data['output_spectrum'].float().cuda(args.gpu, non_blocking=True)

        # Forward pass with mixed precision (autocast)
        with amp.autocast():
            output = net(inputs)
            loss = criterion(output, target)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Backward pass with scaled gradients for stability in AMP
        scaler.scale(loss).backward()

        # Step the optimizer using the scaled loss
        scaler.step(optimizer)
        scaler.update()

        # Update learning rate scheduler, if applicable
        if args.scheduler in ["cyclic-lr", "one-cycle-lr"]:
            scheduler.step()

        # MSE loss calculation outside of AMP (usually in FP32 for stability)
        with torch.no_grad():
            loss_MSE = criterion_MSE(output, target)
            losses.update(loss_MSE.item(), inputs.size(0))

        # Measure batch processing time
        batch_time.update(time.time() - end)
        end = time.time()

        # Log progress every 400 iterations
        if i % 400 == 0:
            progress.display(i)

    return losses.avg




In [8]:
def validate(dataloader, net, criterion_MSE, args):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    progress = ProgressMeter(len(dataloader), [batch_time, losses], prefix='Validation: ')

    with torch.no_grad():
        end = time.time()
        for i, data in enumerate(dataloader):
            inputs = data['input_spectrum']
            inputs = inputs.float()
            inputs = inputs.cuda(args.gpu)
            target = data['output_spectrum']
            target = target.float()
            target = target.cuda(args.gpu)

            output = net(inputs)

            loss_MSE = criterion_MSE(output, target)
            losses.update(loss_MSE.item(), inputs.size(0))

            batch_time.update(time.time() - end)
            end = time.time()

            if i % 400 == 0:
                progress.display(i)

    return losses.avg

In [9]:
def train_noKmeans(args):

    if args.seed is not None:
        random.seed(args.seed)
        torch.manual_seed(args.seed)
        cudnn.deterministic = True

    if args.dist_url == "env://" and args.world_size == -1:
        args.world_size = int(os.environ["WORLD_SIZE"])

    args.distributed = args.world_size > 1 or args.multiprocessing_distributed

    ngpus_per_node = torch.cuda.device_count()

    gpu = args.gpu

    if args.gpu is not None:
        print("Use GPU: {} for training".format(args.gpu))

    if args.distributed:
        if args.dist_url == "env://" and args.rank == -1:
            args.rank = int(os.environ["RANK"])
        if args.multiprocessing_distributed:
            args.rank = args.rank * ngpus_per_node + gpu
        dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                world_size=args.world_size, rank=args.rank)

    # ----------------------------------------------------------------------------------------
    # Create model(s) and send to device(s)
    # ----------------------------------------------------------------------------------------

    net = TransUNet1D(in_channels=1, out_channels=1, embed_dim=64, num_heads=4, depth=4, patch_size=10).float()



    if args.distributed:
        if args.gpu is not None:
            torch.cuda.set_device(args.gpu)
            args.batch_size = int(args.batch_size / ngpus_per_node)
            args.workers = int((args.workers + ngpus_per_node - 1) / ngpus_per_node)

            net.cuda(args.gpu)
            net = torch.nn.parallel.DistributedDataParallel(net, device_ids=[args.gpu])
        else:
            net.cuda(args.gpu)
            net = torch.nn.parallel.DistributedDataParallel(net)
    elif args.gpu is not None:
        torch.cuda.set_device(args.gpu)
        net.cuda(args.gpu)
    else:
        net.cuda(args.gpu)
        net = torch.nn.parallel.DistributedDataParallel(net)

    # ----------------------------------------------------------------------------------------
    # Define dataset path and data splits
    # ----------------------------------------------------------------------------------------
    Input_Data = scipy.io.loadmat("Dataset/Train_Inputs.mat")
    Output_Data = scipy.io.loadmat("Dataset/Train_Outputs.mat")

    Input = Input_Data['Train_Inputs']
    Output = Output_Data['Train_Outputs']

    spectra_num = len(Input)

    train_split = round(0.9 * spectra_num)
    val_split = round(0.1 * spectra_num)

    input_train = Input[:train_split]
    input_val = Input[train_split:train_split+val_split]

    output_train = Output[:train_split]
    output_val = Output[train_split:train_split+val_split]

    # ----------------------------------------------------------------------------------------
    # Create datasets (with augmentation) and dataloaders
    # ----------------------------------------------------------------------------------------
    Raman_Dataset_Train = RamanDataset(input_train, output_train, batch_size = args.batch_size, spectrum_len = args.spectrum_len,
                                   spectrum_shift=0.1, spectrum_window = False, horizontal_flip = False, mixup = True)

    Raman_Dataset_Val = RamanDataset(input_val, output_val, batch_size = args.batch_size, spectrum_len = args.spectrum_len)

# From here down per fold
    train_loader = DataLoader(Raman_Dataset_Train, batch_size = args.batch_size, shuffle = False, num_workers = 0, pin_memory = True)
    val_loader = DataLoader(Raman_Dataset_Val, batch_size = args.batch_size, shuffle = False, num_workers = 0, pin_memory = True)

    # ----------------------------------------------------------------------------------------
    # Define criterion(s), optimizer(s), and scheduler(s)
    # ----------------------------------------------------------------------------------------
    criterion = nn.L1Loss().cuda(args.gpu)
    criterion_MSE = nn.MSELoss().cuda(args.gpu)
    if args.optimizer == "sgd":
        optimizer = optim.SGD(net.parameters(), lr = args.lr)
    elif args.optimizer == "adamW":
        optimizer = optim.AdamW(net.parameters(), lr = args.lr)
    else: # Adam
        optimizer = optim.Adam(net.parameters(), lr = args.lr)

    if args.scheduler == "decay-lr":
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.2)
    elif args.scheduler == "multiplicative-lr":
        lmbda = lambda epoch: 0.985
        scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)
    elif args.scheduler == "cyclic-lr":
        scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr = args.base_lr, max_lr = args.lr, mode = 'triangular2', cycle_momentum = False)
    elif args.scheduler == "one-cycle-lr":
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr = args.lr, steps_per_epoch=len(train_loader), epochs=args.epochs, cycle_momentum = False)
    else: # constant-lr
        scheduler = None

    print('Started Training')
    print('Training Details:')
    print('Network:         {}'.format(args.network))
    print('Epochs:          {}'.format(args.epochs))
    print('Batch Size:      {}'.format(args.batch_size))
    print('Optimizer:       {}'.format(args.optimizer))
    print('Scheduler:       {}'.format(args.scheduler))
    print('Learning Rate:   {}'.format(args.lr))
    print('Spectrum Length: {}'.format(args.spectrum_len))

    DATE = datetime.datetime.now().strftime("%Y_%m_%d")

    formatted_lr = '{:_.6f}'.format(float(args.lr)).rstrip('0').rstrip('.')
    losses_dir = "losses/{}_{}_{}_{}_{}.csv".format(DATE, args.optimizer, args.scheduler, formatted_lr, args.network)
    models_dir = "{}_{}_{}_{}_{}.pt".format(DATE, args.optimizer, args.scheduler, formatted_lr, args.network)

    df = pd.DataFrame(columns=['epoch', 'train_loss', 'val_loss'])

    # Early stopping
    patience = args.patience if hasattr(args, 'patience') else 10  # Default patience of 10 epochs
    best_val_loss = float('inf')
    epochs_no_improve = 0


    for epoch in range(args.epochs):
        train_loss = train(train_loader, net, optimizer, scheduler, criterion, criterion_MSE, epoch, args)
        val_loss = validate(val_loader, net, criterion_MSE, args)
        if args.scheduler == "decay-lr" or args.scheduler == "multiplicative-lr":
            scheduler.step()

        print("Epoch: ", epoch)
        print("Train Loss: ", train_loss)
        print("Val Loss: ", val_loss)
        new_row = pd.DataFrame({'epoch': [epoch], 'train_loss': [train_loss], 'val_loss': [val_loss]})

        df = pd.concat([df, new_row], ignore_index=True)

        # Early Stopping Logic
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered. No improvement in validation loss for {patience} epochs. Finished at epoch {epoch}")
                break

        torch.cuda.empty_cache()

    torch.save(net.state_dict(), models_dir)
    df.to_csv(losses_dir, index=False)
    print('Finished Training')

In [10]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [11]:
%ls

dataset.py  model.py  ResUNet.pt  utilities.py


In [12]:
%cd ../..

/content


In [13]:
%cd drive/My\ Drive/Colab\ Notebooks/DeepeR-master/Raman Spectral Denoising

/content/drive/My Drive/Colab Notebooks/DeepeR-master/Raman Spectral Denoising


In [ ]:
# Default args from original code
#Namespace(workers=0, epochs=2, start_epoch=0, batch_size=256, network='ResUNet', optimizer='adam', lr=0.0005, base_lr=5e-06, scheduler='one-cycle-lr', batch_norm=True, spectrum_len=500, seed=None, gpu=0, world_size=-1, rank=-1, dist_url='tcp://224.66.41.62:23456', dist_backend='nccl', multiprocessing_distributed=False)

class Arguments:
    pass

args = Arguments()
args.workers = 0
args.epochs = 500
args.start_epoch = 0
args.batch_size = 256
args.network = "TransUNet1D"
args.optimizer = "adamW"
args.lr = 1e-4
args.base_lr = 5e-6
args.scheduler = "one-cycle-lr"
args.batch_norm = True
args.spectrum_len = 500
args.seed = None
args.gpu = 0
args.world_size = -1
args.rank = -1
args.dist_url = "tcp://224.66.41.62:23456"
args.dist_backend = "nccl"
args.multiprocessing_distributed = False
args.patience = 10


args.epochs=500
train_noKmeans(args)

Use GPU: 0 for training


In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
def calc_psnr(output, target):
    psnr = 0.
    mse = nn.MSELoss()(output, target)
    psnr = 10 * math.log10(torch.max(output)/mse)
    return psnr

def calc_ssim(output, target):
    ssim = 0.
    output = output.cpu().detach().numpy()
    target = target.cpu().detach().numpy()

    if output.ndim == 4:
        for i in range(output.shape[0]):
            output_i = np.squeeze(output[i,:,:,:])
            output_i = np.moveaxis(output_i, 0, -1)
            target_i = np.squeeze(target[i,:,:,:])
            target_i = np.moveaxis(target_i, 0, -1)
            batch_size = output.shape[0]
            ssim += sk_ssim(output_i, target_i, data_range = output_i.max() - target_i.max(), multichannel=True)
    else:
        output_i = np.squeeze(output)
        output_i = np.moveaxis(output_i, 0, -1)
        target_i = np.squeeze(target)
        target_i = np.moveaxis(target_i, 0, -1)
        batch_size = 1
        ssim += sk_ssim(output_i, target_i, data_range = output_i.max() - target_i.max(), multichannel=True)

    ssim = ssim / batch_size
    return ssim

In [ ]:
# Testing
def evaluate(dataloader, net, args):
    losses = AverageMeter('Loss', ':.4e')
    psnr = AverageMeter('PSNR', ':.4f')
    ssim = AverageMeter('SSIM', ':.4f')
    SG_loss = AverageMeter('Savitzky-Golay Loss', ':.4e')

    net.eval()

    MSE_SG = []

    with torch.no_grad():
        for i, data in enumerate(dataloader):
            x = data['input_spectrum']
            inputs = x.float()
            inputs = inputs.cuda(args.gpu)
            y = data['output_spectrum']
            target = y.float()
            target = target.cuda(args.gpu)

            x = np.squeeze(x.numpy())
            y = np.squeeze(y.numpy())

            output = net(inputs)
            loss = nn.MSELoss()(output, target)

            x_out = output.cpu().detach().numpy()
            x_out = np.squeeze(x_out)

            SGF_1_9 = scipy.signal.savgol_filter(x,9,1)
            MSE_SGF_1_9 = np.mean(np.mean(np.square(np.absolute(y - (SGF_1_9 - np.reshape(np.amin(SGF_1_9, axis = 1), (len(SGF_1_9),1)))))))
            MSE_SG.append(MSE_SGF_1_9)

            psnr_batch = calc_psnr(output, target)
            psnr.update(psnr_batch, inputs.size(0))
            ssim_batch = calc_ssim(output, target)
            ssim.update(ssim_batch, inputs.size(0))

            losses.update(loss.item(), inputs.size(0))

        print("Neural Network MSE: {}".format(losses.avg))
        print("Neural Network PSNR: {}".format(psnr.avg))
        print("Neural Network SSIM: {}".format(ssim.avg))
        print("Savitzky-Golay MSE: {}".format(np.mean(np.asarray(MSE_SG))))
        print("Neural Network performed {0:.2f}x better than Savitzky-Golay".format(np.mean(np.asarray(MSE_SG))/losses.avg))

    return losses.avg, psnr.avg, ssim.avg, MSE_SG

In [ ]:
def main_test(args):
    gpu = args.gpu
    if args.seed is not None:
        random.seed(args.seed)
        torch.manual_seed(args.seed)
        cudnn.deterministic = True

    if args.dist_url == "env://" and args.world_size == -1:
        args.world_size = int(os.environ["WORLD_SIZE"])

    args.distributed = args.world_size > 1 or args.multiprocessing_distributed

    ngpus_per_node = torch.cuda.device_count()


    if args.gpu is not None:
        print("Use GPU: {} for testing".format(args.gpu))

    if args.distributed:
        if args.dist_url == "env://" and args.rank == -1:
            args.rank = int(os.environ["RANK"])
        if args.multiprocessing_distributed:
            args.rank = args.rank * ngpus_per_node + gpu
        dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                world_size=args.world_size, rank=args.rank)

    # ----------------------------------------------------------------------------------------
    # Create model(s) and send to device(s)
    # ----------------------------------------------------------------------------------------
    if args.network == "UNetPlusPlus1D":
      net = UNetPlusPlus1D().float()
    elif args.network == "UNet1D":
      net = UNet1D().float()
    elif args.network == "AttentionUNet":
      net = AttentionUNet().float()
    net.load_state_dict(torch.load(args.model))

    if args.distributed:
        if args.gpu is not None:
            torch.cuda.set_device(args.gpu)
            args.batch_size = int(args.batch_size / ngpus_per_node)
            args.workers = int((args.workers + ngpus_per_node - 1) / ngpus_per_node)

            net.cuda(args.gpu)
            net = torch.nn.parallel.DistributedDataParallel(net, device_ids=[args.gpu])
        else:
            net.cuda(args.gpu)
            net = torch.nn.parallel.DistributedDataParallel(net)
    elif args.gpu is not None:
        torch.cuda.set_device(args.gpu)
        net.cuda(args.gpu)
    else:
        net.cuda(args.gpu)
        net = torch.nn.parallel.DistributedDataParallel(net)

    # ----------------------------------------------------------------------------------------
    # Define dataset path and data splits
    # ----------------------------------------------------------------------------------------
    Input_Data = scipy.io.loadmat("Dataset/Test_Inputs.mat")
    Output_Data = scipy.io.loadmat("Dataset/Test_Outputs.mat")

    Input = Input_Data['Test_Inputs']
    Output = Output_Data['Test_Outputs']

    # ----------------------------------------------------------------------------------------
    # Create datasets (with augmentation) and dataloaders
    # ----------------------------------------------------------------------------------------
    Raman_Dataset_Test = RamanDataset(Input, Output, batch_size = args.batch_size, spectrum_len = args.spectrum_len)

    test_loader = DataLoader(Raman_Dataset_Test, batch_size = args.batch_size, shuffle = False, num_workers = 0, pin_memory = True)

    # ----------------------------------------------------------------------------------------
    # Evaluate
    # ----------------------------------------------------------------------------------------
    MSE_NN, PSNR_NN, SSIM_NN, MSE_SG = evaluate(test_loader, net, args)

In [ ]:
class Arguments:
    pass

args = Arguments()
args.workers = 0
args.batch_size = 256
args.spectrum_len = 500
args.seed = None
args.gpu = 0
args.world_size = -1
args.rank = -1
args.dist_url = "tcp://224.66.41.62:23456"
args.dist_backend = "nccl"
args.multiprocessing_distributed = False
args.batch_norm = True
args.network = "UNet1D"
args.model = "2024_10_09_adamW_one-cycle-lr_0.001_UNet1D.pt"


main_test(args)

Use GPU: 0 for testing


<ipython-input-20-cf748de30bb6>:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(args.model))


Neural Network MSE: 0.0023067386205582094
Neural Network PSNR: 26.385451591812636
Neural Network SSIM: 0.2742282462580637
Savitzky-Golay MSE: 0.027660622850368643
Neural Network performed 11.99x better than Savitzky-Golay


In [ ]:
file_name = 'losses/training_logs.csv'

df = pd.read_csv(file_name)

# Plotting train_loss and val_loss
plt.plot(df['epoch'], df['train_loss'], label='Train Loss', marker='o')
plt.plot(df['epoch'], df['val_loss'], label='Validation Loss', marker='o')

# Adding labels and title
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train and Validation Loss vs Epoch')

# Adding a legend to distinguish the two lines
plt.legend()

# Show the plot
plt.show()
